In [1]:
import sys
import json
import shutil
import os

sys.path.insert(0, '../src') # add library code to path
from geospatial import *

In [2]:
df = read_gis()

In [3]:
df.head()

,AIANHHA,AITSCEA,ALAND10,ANRCA,APREC,AREA,AWATER10,BASICCAR,BLKGRPA,BLOCKA,...,TRACTCE10,TRUSTA,TTRACTA,UAA,UACE10,UATYP10,UR10,URBRURALA,YEAR,ZCTA5A
0,9999,999,19628,99999,NORTHEAST,1.376417e+07,0,11A25,1,1008,...,183104,9999,999999,51445,51445,U,U,U,2010,90042
1,9999,999,3418,99999,NORTHEAST,1.439270e+07,0,11A17,3,3009,...,186100,9999,999999,51445,51445,U,U,U,2010,90065
2,9999,999,18866,99999,NORTHEAST,2.387249e+07,0,11A1,1,1016,...,188100,9999,999999,51445,51445,U,U,U,2010,90039
3,9999,999,19666,99999,NORTHEAST,2.387249e+07,0,11A1,3,3007,...,188100,9999,999999,51445,51445,U,U,U,2010,90039
4,9999,999,11865,99999,PACIFIC,1.126278e+07,0,14A49,3,3019,...,275603,9999,999999,51445,51445,U,U,U,2010,90066


In [4]:
arrests = pd.read_csv('../data/cleaned/arrests-processed.csv')

In [5]:
arrests.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Year,predPol Deployed,Total
0,191811472,2019-05-03,1700.0,18,Southeast,1802,23,F,B,NaN,NaN,M,653.22 PC,NaN,2019,1,1
1,5614161,2019-04-29,1040.0,8,West LA,842,41,M,H,3.0,Robbery,F,211PC,ROBBERY,2019,1,1
2,5806609,2019-11-23,1830.0,4,Hollenbeck,457,23,F,H,3.0,Robbery,F,211PC,ROBBERY,2019,1,1
3,5615197,2019-04-30,615.0,6,Hollywood,663,27,M,O,5.0,Burglary,F,459PC,BURGLARY,2019,1,1
4,5615701,2019-04-30,1100.0,9,Van Nuys,901,2,F,H,NaN,NaN,D,300(B)WIC,NaN,2019,1,1


In [8]:
df.columns

Index(['AIANHHA', 'AITSCEA', 'ALAND10', 'ANRCA', 'APREC', 'AREA', 'AWATER10',
       'BASICCAR', 'BLKGRPA', 'BLOCKA',
       ...
       'TRACTCE10', 'TRUSTA', 'TTRACTA', 'UAA', 'UACE10', 'UATYP10', 'UR10',
       'URBRURALA', 'YEAR', 'ZCTA5A'],
      dtype='object', length=146)

In [16]:
race_map = {
    "H7Z001": "Total",
    "H7Z003": "White",
    "H7Z004": "Black or African American",
    "H7Z005": "American Indian and Alaska Native",
    "H7Z006": "Asian",
    "H7Z007": "Native Hawaiian and Other Pacific Islander",
    "H7Z008": "Other",
    "H7Z010": "Hispanic or Latino"
}

In [17]:
renamed_cols = df.rename(columns=race_map)

In [18]:
for col in renamed_cols:
    if col.startswith('H7'):
        renamed_cols.drop(col, axis=1, inplace=True)

In [19]:
renamed_cols.columns

Index(['AIANHHA', 'AITSCEA', 'ALAND10', 'ANRCA', 'APREC', 'AREA', 'AWATER10',
       'BASICCAR', 'BLKGRPA', 'BLOCKA', 'BLOCKCE10', 'BUREAU', 'CBSAA', 'CDA',
       'CNECTAA', 'CONCITA', 'COUNTY', 'COUNTYA', 'COUNTYFP10', 'COUSUBA',
       'CSAA', 'DIVISIONA', 'FUNCSTAT10', 'GEOID10', 'GISJOIN', 'GISJOIN_1',
       'Total', 'White', 'Black or African American',
       'American Indian and Alaska Native', 'Asian',
       'Native Hawaiian and Other Pacific Islander', 'Other',
       'Hispanic or Latino', 'INTPTLAT10', 'INTPTLON10', 'Join_Count',
       'METDIVA', 'MTFCC10', 'NAME', 'NAME10', 'NECTAA', 'NECTADIVA',
       'OBJECTID', 'ORIG_FID', 'PERIMETER', 'PLACEA', 'PREC', 'REGIONA',
       'REPDIST', 'RES_ONLYA', 'SABINSA', 'SDELMA', 'SDSECA', 'SDUNIA',
       'SHAPE', 'SLDLA', 'SLDUA', 'STATE', 'STATEA', 'STATEFP10', 'SUBMCDA',
       'Shape__Area', 'Shape__Area_1', 'Shape__Area_2', 'Shape__Length',
       'Shape__Length_1', 'Shape__Length_2', 'TBLKGRPA', 'TRACTA', 'TRACTCE10',
      

In [51]:
census_div = renamed_cols[['Total', 'White', 'Black or African American',
       'American Indian and Alaska Native', 'Asian',
       'Native Hawaiian and Other Pacific Islander', 'Other',
       'Hispanic or Latino']].sum()
census_div

Total                                         3789918
White                                         1086014
Black or African American                      347262
American Indian and Alaska Native                6583
Asian                                          419961
Native Hawaiian and Other Pacific Islander       4298
Other                                           12052
Hispanic or Latino                            1837450
dtype: int64

In [53]:
RACE_MAP = {'B': 'Black','H': 'Hispanic or Latino','O': 'Other','W': 'White','A': 'Asian','K': 'Asian','F': 'Asian','V': 'Asian','G': 'Native Hawaiian and Other Pacific Islander','C': 'Asian','X': 'Other','P': 'Native Hawaiian and Other Pacific Islander','S': 'Native Hawaiian and Other Pacific Islander','J': 'Asian','I': 'American Indian and Alaska Native','U': 'Native Hawaiian and Other Pacific Islander','L': 'Asian','Z': 'Asian','D': 'Asian'}
arrests['Descent Code'] = arrests['Descent Code'].map(RACE_MAP)

In [54]:
pd.DataFrame(arrests.loc[arrests.Year != 2020].groupby('Descent Code').apply(lambda x: len(x)), columns=['Arrest Rate'])

,Arrest Rate
Descent Code,
American Indian and Alaska Native,284
Asian,8445
Black,380745
Hispanic or Latino,617828
Native Hawaiian and Other Pacific Islander,250
Other,68142
White,248685
